## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Preparación de los datos

---

En esta libreta se realizan las transforamciones sobre los datasets del escenario en función 
de los resultados de la verificación de calidad de datos. 

---

In [2]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

In [3]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_mc.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_mc.csv", sep=";")
display(df_tarjetas.head(1))

df_integrado = pd.read_csv("../../data/processed/datos_integrados.csv", sep=";")
display(df_integrado.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,0,N,0.0,True


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,708082083.0,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,regla_pct_ingresos
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,...,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0,ok


---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


----

Selección de datos

In [4]:
# Se establece qué columnas se eliminan

col_eliminar_creditos = ['completitud_fila', 'situacion_vivienda_ok']
col_eliminar_tarjetas = ['nivel_tarjeta']

# Se ejecuta la operación

df_creditos.drop(col_eliminar_creditos, inplace=True, axis=1)
df_tarjetas.drop(col_eliminar_tarjetas, inplace=True, axis=1)

In [5]:
print("Vista del dataset de datos de créditos:")
display(df_creditos.head(1))

print("Vista del dataset de datos de tarjetas:")
display(df_tarjetas.head(1))

Vista del dataset de datos de créditos:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,0,N


Vista del dataset de datos de tarjetas:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo
0,708082083.0,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,111.0,3.0


Limpieza de los datos (filtrado a nivel de filas)

In [6]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_creditos.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_pct_ingresos], axis=1)
df_creditos.head(5)  

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,0,N,ok
1,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,15.99,1,Y,ok
2,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,12.72,0,Y,ok
3,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,8.00,0,N,ok
4,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,7.74,0,N,ok


In [7]:
df_creditos[df_creditos['regla_pct_ingresos'] == 'err'].shape[0]

15

In [8]:
# Se filtran las filas con algún error detectado
print(f"Filas antes del filtro: {df_creditos.shape[0]}")

temp = df_creditos[df_creditos['edad'] < 90]

# Otro filtro posible: por la regla de negocio agregada

temp_c = temp[temp['regla_pct_ingresos'] == 'ok']

print(f"Filas después del filtro: {temp_c.shape[0]}")

Filas antes del filtro: 10127
Filas después del filtro: 10108


Integración de datos

In [9]:
df_integrado = pd.merge(temp_c, df_tarjetas, on='id_cliente', how='inner')
coincidencias = df_integrado.shape[0]

print(f"Filas del dataset integrado con los filtros realizados: {coincidencias}")

Filas del dataset integrado con los filtros realizados: 10108


In [10]:
print(f"Cantidad de columnas del dataset integrado: {df_integrado.shape[1]}")

Cantidad de columnas del dataset integrado: 22


#### Transformación de atributos

Atributos nominales que se modifican los valores

In [11]:
# Columna: estado_civil
cambios_estado_civil = {
    'CASADO' : 'C',
    'SOLTERO' : 'S',
    'DESCONOCIDO' : 'N',
    'DIVORCIADO' : 'D',
}

estado_civil_N = df_integrado.loc[:, ('estado_civil')].map(cambios_estado_civil).rename('estado_civil_N')

# Columna: estado_credito
cambios_estado_credito = {
    0: 'C',
    1 : 'P',
}

estado_credito_N = df_integrado.loc[:, ('estado_credito')].map(cambios_estado_credito).rename('estado_credito_N')

df_integrado = pd.concat([estado_civil_N, estado_credito_N, df_integrado], axis=1)
df_integrado.head()

# Sobre este resultado será necesario eliminar las columnas auxiliares

,estado_civil_N,estado_credito_N,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,...,regla_pct_ingresos,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo
0,C,C,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,...,ok,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,111.0,3.0
1,S,P,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,...,ok,45.0,SOLTERO,PASIVO,992.0,M,3421.0,DESCONOCIDO,21.0,0.0
2,D,C,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,...,ok,38.0,DIVORCIADO,PASIVO,1447.0,M,8258.0,POSGRADO_COMPLETO,23.0,3.0
3,S,C,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,...,ok,24.0,SOLTERO,ACTIVO,3940.0,F,1438.3,UNIVERSITARIO_INCOMPLETO,82.0,2.0
4,C,C,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,...,ok,41.0,CASADO,ACTIVO,4369.0,F,3128.0,UNIVERSITARIO_INCOMPLETO,59.0,2.0


Atributos numéricos que se discretizan

In [12]:
df_integrado['operaciones_ult_12m'].describe()

count    10108.000000
mean        64.892659
std         23.465252
min         10.000000
25%         45.000000
50%         67.000000
75%         81.000000
max        139.000000
Name: operaciones_ult_12m, dtype: float64

In [13]:
# Antiguedad del empleado

etiquetas_a_e = ['menor_5', '5_a_10', 'mayor_10']
rangos_a_e = [0, 4, 10, 50]
valor_para_nan = 'NA'
antiguedad_empleados_N = pd.cut(df_integrado['antiguedad_empleado'], 
                                bins=rangos_a_e, 
                                labels=etiquetas_a_e,
                                right=False).cat.add_categories(valor_para_nan).fillna(valor_para_nan)

antiguedad_empleados_N.value_counts()

# edad

etiquetas_e = ['menor_25', '25_a_30']
rangos_e = [0, 24, 50]
edad_N = pd.cut(df_integrado['edad'], 
                                bins=rangos_e, 
                                labels=etiquetas_e)

edad_N.value_counts()

# pct_ingreso

etiquetas_p_i = ['hasta_20', '20_a_40', '40_a_60', 'mayor_60']
rangos_p_i = [0, 0.19, 0.39, 0.60, 0.99]
pct_ingreso_N = pd.cut(df_integrado['pct_ingreso'], 
                                bins=rangos_p_i, 
                                labels=etiquetas_p_i)

pct_ingreso_N.value_counts()

# ingresos

etiquetas_i = ['hasta_20k', '20k_a_50k', '50k_a_100k', 'mayor_100k']
rangos_i = [0, 19999, 49999, 99999, 999999]
ingresos_N = pd.cut(df_integrado['ingresos'], 
                                bins=rangos_i, 
                                labels=etiquetas_i)

ingresos_N.value_counts()

# tasa_interes

etiquetas_t_i = ['hasta_7p', '7p_a_15p', '15p_a_20p', 'mayor_20p']
rangos_t_i = [0, 6.99, 14.99, 19.99, 100]
tasa_interes_N = pd.cut(df_integrado['tasa_interes'], 
                                bins=rangos_t_i, 
                                labels=etiquetas_t_i)

tasa_interes_N.value_counts()

# antiguedad_cliente

etiquetas_a_c = ['menor_2y', '2y_a_4y', 'mayor_4y']
rangos_a_c = [0, 24, 48, 100]
antiguedad_cliente_N = pd.cut(df_integrado['antiguedad_cliente'], 
                                bins=rangos_a_c, 
                                labels=etiquetas_a_c)

antiguedad_cliente_N.value_counts()

# limite_credito_tc

etiquetas_l_tc = ['menor_3k', '3k_a_5k', '5k_a_10k', 'mayor_10k']
rangos_l_tc = [0, 2999, 4999, 9999, 100000]
limite_credito_tc_N = pd.cut(df_integrado['limite_credito_tc'], 
                                bins=rangos_l_tc, 
                                labels=etiquetas_l_tc)

limite_credito_tc_N.value_counts()

# gastos_ult_12m

etiquetas_g_u12 = ['menor_1k', '2k_a_4k', '4k_a_6k', '6k_a_8k', '8k_a_10k', 'mayor_10k']
rangos_g_u12 = [0, 999, 3999, 5999, 7999, 9999, 100000]
gastos_ult_12m_N = pd.cut(df_integrado['gastos_ult_12m'], 
                                bins=rangos_g_u12, 
                                labels=etiquetas_g_u12)

gastos_ult_12m_N.value_counts()

# operaciones_ult_12m

etiquetas_o_u12 = ['menor_15', '15_a_30', '30_a_50', '50_a_75', '75_a_100', 'mayor_100']
rangos_o_u12 = [0, 14, 29, 49, 74, 99, 1000]
operaciones_ult_12m_N = pd.cut(df_integrado['operaciones_ult_12m'], 
                                bins=rangos_o_u12, 
                                labels=etiquetas_o_u12)

operaciones_ult_12m_N.value_counts()

operaciones_ult_12m
50_a_75      3393
75_a_100     3004
30_a_50      2413
mayor_100     687
15_a_30       587
menor_15       24
Name: count, dtype: int64

In [14]:
df_integrado.columns

Index(['estado_civil_N', 'estado_credito_N', 'id_cliente', 'edad',
       'importe_solicitado', 'duracion_credito', 'antiguedad_empleado',
       'situacion_vivienda', 'ingresos', 'objetivo_credito', 'pct_ingreso',
       'tasa_interes', 'estado_credito', 'falta_pago', 'regla_pct_ingresos',
       'antiguedad_cliente', 'estado_civil', 'estado_cliente',
       'gastos_ult_12m', 'genero', 'limite_credito_tc', 'nivel_educativo',
       'operaciones_ult_12m', 'personas_a_cargo'],
      dtype='object')

In [15]:
col_eliminar_final = [
              'edad',
              'antiguedad_empleado',
              'antiguedad_cliente', 
              'ingresos',
              'pct_ingreso', 
              'tasa_interes',
              'regla_pct_ingresos',
              'gastos_ult_12m', 
              'limite_credito_tc', 
              'operaciones_ult_12m',
              'id_cliente']

df_integrado.drop(col_eliminar_final, inplace=True, axis=1)


df_final = pd.concat([operaciones_ult_12m_N, gastos_ult_12m_N, limite_credito_tc_N, antiguedad_cliente_N, tasa_interes_N, ingresos_N, pct_ingreso_N, antiguedad_empleados_N, edad_N, df_integrado], axis=1)
df_final.head(5)


,operaciones_ult_12m,gastos_ult_12m,limite_credito_tc,antiguedad_cliente,tasa_interes,ingresos,pct_ingreso,antiguedad_empleado,edad,estado_civil_N,...,duracion_credito,situacion_vivienda,objetivo_credito,estado_credito,falta_pago,estado_civil,estado_cliente,genero,nivel_educativo,personas_a_cargo
0,mayor_100,mayor_10k,3k_a_5k,2y_a_4y,hasta_7p,50k_a_100k,hasta_20,5_a_10,menor_25,C,...,3,HIPOTECA,INVERSIONES,0,N,CASADO,ACTIVO,F,SECUNDARIO_COMPLETO,3.0
1,15_a_30,menor_1k,3k_a_5k,2y_a_4y,15p_a_20p,20k_a_50k,hasta_20,menor_5,menor_25,S,...,2,ALQUILER,MEJORAS_HOGAR,1,Y,SOLTERO,PASIVO,M,DESCONOCIDO,0.0
2,15_a_30,2k_a_4k,5k_a_10k,2y_a_4y,7p_a_15p,20k_a_50k,20_a_40,5_a_10,menor_25,D,...,2,OTROS,EDUCACIÓN,0,Y,DIVORCIADO,PASIVO,M,POSGRADO_COMPLETO,3.0
3,75_a_100,2k_a_4k,menor_3k,menor_2y,7p_a_15p,20k_a_50k,20_a_40,menor_5,25_a_30,S,...,4,ALQUILER,INVERSIONES,0,N,SOLTERO,ACTIVO,F,UNIVERSITARIO_INCOMPLETO,2.0
4,50_a_75,4k_a_6k,3k_a_5k,2y_a_4y,7p_a_15p,50k_a_100k,20_a_40,menor_5,25_a_30,C,...,2,HIPOTECA,EDUCACIÓN,0,N,CASADO,ACTIVO,F,UNIVERSITARIO_INCOMPLETO,2.0


In [16]:
col_eliminar_final_N = [
              'estado_credito',
              'estado_civil',]

df_final.drop(col_eliminar_final_N, inplace=True, axis=1)

Exportación de metadatos

In [17]:
import dtale as dt

informe = dt.show(df_final)
informe.open_browser()

Exportación del dataset

In [18]:
df_final.to_csv("../../data/final/datos_finales.csv", sep=';', index=False)

2023-12-10 12:50:37,148 - ERROR    - Exception occurred while processing request: Cannot setitem on a Categorical with a new category (Missing), set the categories first
Traceback (most recent call last):
  File "c:\Users\juan_\miniconda3\envs\viu\Lib\site-packages\dtale\views.py", line 119, in _handle_exceptions
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\juan_\miniconda3\envs\viu\Lib\site-packages\dtale\views.py", line 3102, in get_column_analysis
    return jsonify(**analysis.build())
                     ^^^^^^^^^^^^^^^^
  File "c:\Users\juan_\miniconda3\envs\viu\Lib\site-packages\dtale\column_analysis.py", line 142, in build
    return_data, code = self.analysis.build(self)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\juan_\miniconda3\envs\viu\Lib\site-packages\dtale\column_analysis.py", line 697, in build
    parent.data[parent.selected_col].fillna("Missing"), self.cleaners
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^